<a href="https://colab.research.google.com/github/mostafa-ja/cloud-computing/blob/main/FatTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import k from user's input
k = 4

#open the output file
f = open("output.txt","w")


#Calculating the pods, servers, core switches, aggregation switches

pods_n = int(k)
AggrSwitch_in_pod = int(k/2)
AggrSwitch_n = int(k*AggrSwitch_in_pod)
EdgeSwitch_in_pod = int(k/2)
EdgeSwitch_n = int(k*EdgeSwitch_in_pod)
Servers_in_pod = int(EdgeSwitch_in_pod*(k/2))
Servers_n = int(Servers_in_pod*k)
CoreSwitch_n = int((k/2)**2)

AllSwitch_n = EdgeSwitch_n + AggrSwitch_n + CoreSwitch_n
allElements_n = AllSwitch_n + Servers_n

# we address switches and servers with numbers from bottom to top
Servers_range_start = 0  #include zero
Servers_range_end = (Servers_range_start + Servers_n) #exclude the end number
EdgeSwitch_range_start = Servers_range_end
EdgeSwitch_range_end = (EdgeSwitch_range_start + EdgeSwitch_n)
AggrSwitch_range_start = EdgeSwitch_range_end
AggrSwitch_range_end = (EdgeSwitch_range_end + AggrSwitch_n)
CoreSwitch_range_start = AggrSwitch_range_end
CoreSwitch_range_end = (AggrSwitch_range_end + CoreSwitch_n)

f.write(f'Servers_range = [{Servers_range_start},{Servers_range_end}) \n')
f.write(f'EdgeSwitch_range = [{EdgeSwitch_range_start},{EdgeSwitch_range_end}) \n')
f.write(f'AggrSwitch_range = [{AggrSwitch_range_start},{AggrSwitch_range_end}) \n')
f.write(f'CoreSwitch_range = [{CoreSwitch_range_start},{CoreSwitch_range_end}) \n')
f.write('-------------- \n')

16

In [2]:
#connected list used when we want to keep track of connection and which are 1
connected= []

In [3]:
# program goes stage by stage bottom up, first we create links between EdgeSwitch and Servers
server_address = Servers_range_start
for EdgeSwitch_address in range(EdgeSwitch_range_start,EdgeSwitch_range_end):
  for server in range(int(k/2)):   #each EdgeSwitch connects to (k/2) servers
    out = (f'{EdgeSwitch_address:5d} \t {server_address:5d} \t 1 \n')
    f.write(out)
    connected.append((EdgeSwitch_address,server_address))
    out = (f'{server_address:5d} \t {EdgeSwitch_address:5d} \t 1 \n')
    f.write(out)
    connected.append((server_address,EdgeSwitch_address))

    server_address += 1


In [4]:
#connection between Aggregation switches and edge switches , from top to bottom

for pod in range(pods_n):
  AggrSwitch_address = int(AggrSwitch_range_start + k/2 * pod)
  for aggr in range(int(k/2)):    #each pod has k/2 AggrSwitches
    EdgeSwitch_address = int(EdgeSwitch_range_start + k/2 * pod)
    for edge in range(int(k/2)):  #each AggrSwitch connects to k/2 EdgeSwitches
      out = (f'{AggrSwitch_address:5d} \t {EdgeSwitch_address:5d} \t 1 \n')
      f.write(out)
      connected.append((AggrSwitch_address,EdgeSwitch_address))
      out = (f'{EdgeSwitch_address:5d} \t {AggrSwitch_address:5d} \t 1 \n')
      f.write(out)
      connected.append((EdgeSwitch_address,AggrSwitch_address))
      EdgeSwitch_address += 1

    AggrSwitch_address +=1

In [5]:
#connection between Aggregation switches and Core switches , from bottom to top
# we know that in each pot , Aggregation switches connect to all cores , each one with k/2 connection
for pod in range(pods_n):
  AggrSwitch_address = int(AggrSwitch_range_start + k/2 * pod)
  CoreSwitch_address = CoreSwitch_range_start
  for aggr in range(int(k/2)):    #each pod has k/2 AggrSwitches
    for core in range(int(k/2)):
      out = (f'{AggrSwitch_address:5d} \t {CoreSwitch_address:5d} \t 1 \n')
      f.write(out)
      connected.append((AggrSwitch_address,CoreSwitch_address))
      out = (f'{CoreSwitch_address:5d} \t {AggrSwitch_address:5d} \t 1 \n')
      f.write(out)
      connected.append((CoreSwitch_address,AggrSwitch_address))
      CoreSwitch_address += 1

    AggrSwitch_address +=1

In [6]:
# implementing the zero connections
for i in range(allElements_n):
  for j in range(allElements_n):
    if ((i,j) not in connected) and (i != j):
      out = (f'{i:5d} \t {j:5d} \t 0 \n')
      f.write(out)

In [7]:
f.close()

In [8]:
! pip install pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 592 kB 43.1 MB/s 
     |████████████████████████████████| 1.6 MB 73.2 MB/s 
  Created wheel for pyvis: filename=pyvis-0.3.0-py3-none-any.whl size=600250 sha256=23d02c99535793173bacabe955c254cc329c4972521f415e5a91bbfcfcf344c4
  Stored in directory: /root/.cache/pip/wheels/57/74/32/e404e0d76fbb2dc4e3791b821cfae6a69c1a6677ded99dc8b7
Successfully built pyvis


In [9]:
from pyvis.network import Network
net = Network(height='700px', width='100%')



for i in range(Servers_range_start,Servers_range_end):
  net.add_node(i,label=str(i),group='a')

for i in range(EdgeSwitch_range_start,EdgeSwitch_range_end):
  net.add_node(i,label=str(i),group='b')

for i in range(AggrSwitch_range_start,AggrSwitch_range_end):
  net.add_node(i,label=str(i),group='c')

for i in range(CoreSwitch_range_start,CoreSwitch_range_end):
  net.add_node(i,label=str(i),group='d')

net.add_edges(connected)

#for i,j in connected:
  #net.add_edge(i,j)

#net.repulsion(node_distance=400, spring_length=100)
net.show("Network.html")

In [10]:
net.neighbors(16)

{0, 1, 24, 25}

In [11]:
net.num_edges()

48

In [12]:
net.num_nodes()

36